### 1. Using Light GBM

In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from scipy import sparse as ssp
from sklearn.model_selection import StratifiedKFold
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import roc_auc_score
from sklearn import datasets, linear_model, metrics
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings("ignore")
import time

In [45]:
param = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'learning_rate': 0.05,
    'num_leaves': 255,  
    'max_depth': 7,  
    'min_child_samples':50,  
    'max_bin': 100,  
    'subsample': 0.7,  
    'subsample_freq': 1,  
    'colsample_bytree': 0.7,  
    'min_child_weight': 0,  
    'subsample_for_bin': 200,  
    'min_split_gain': 0,  
    'reg_alpha': 1,  
    'reg_lambda': 1,  
   # 'nthread': 8,
    'verbose': 5,
    'scale_pos_weight':99 
    }
        

In [3]:
Path = ".//CV_dataset//"

In [24]:
def get_function(train_df):
    train_df = pd.get_dummies(train_df,prefix='pin_',dummy_na= True, columns=['catg_pincode_id_ftg'])
    train_df = pd.get_dummies(train_df,prefix='emp_code_',dummy_na= True, columns=['catg_employee_code_id_ftg'])
    train_df = pd.get_dummies(train_df,prefix='supp_id_',dummy_na= True, columns=['catg_supplier_id_ftg'])
    train_df = pd.get_dummies(train_df,prefix='branch_id_',dummy_na= True, columns=['catg_branch_id_ftg'])
    train_df = pd.get_dummies(train_df,prefix='manufacturer_id_',dummy_na= True, columns=['catg_manufacturer_id_ftg'])
    train_df = pd.get_dummies(train_df,prefix='state_id_',dummy_na= True, columns=['catg_state_id_ftg'])

    train_df = pd.get_dummies(train_df,prefix='catg_cns_desc_',dummy_na= True, columns=['PERFORM_CNS.SCORE.DESCRIPTION'])
    train_df = pd.get_dummies(train_df,prefix='catg_emp_type',dummy_na= True, columns=['Employment.Type'])
    train_df['Total_verification_flags'] = train_df['Total_verification_flags'].astype('int8')                
    
    return train_df

In [25]:
pd.DataFrame(features).to_csv('Features.csv')

In [27]:
'loan_default' in features

False

In [43]:
feat_common=  ['disbursed_amount', 'asset_cost', 'ltv', 'Aadhar_flag', 'PAN_flag', 'VoterID_flag',\
                'Driving_flag', 'Passport_flag', 'PERFORM_CNS.SCORE', 'PRI.NO.OF.ACCTS', 'PRI.ACTIVE.ACCTS', 'PRI.OVERDUE.ACCTS',
                'PRI.CURRENT.BALANCE', 'PRI.SANCTIONED.AMOUNT', 'PRI.DISBURSED.AMOUNT', 'SEC.NO.OF.ACCTS',
                'SEC.ACTIVE.ACCTS', 'SEC.OVERDUE.ACCTS', 'SEC.CURRENT.BALANCE',
               'SEC.SANCTIONED.AMOUNT', 'SEC.DISBURSED.AMOUNT', 'PRIMARY.INSTAL.AMT',
               'SEC.INSTAL.AMT', 'NEW.ACCTS.IN.LAST.SIX.MONTHS',
               'DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS', 'NO.OF_INQUIRIES',
               'Age_at_disbursaldate',   'Converted_AVERAGE.ACCT.AGE',
               'Converted_CREDIT.HISTORY.LENGTH','Total_verification_flags',
           'Disbursal_Month','Disbursal_Day'] 

In [ ]:
# Doing K fold crossvalidation
oof_all=[]
feature_importance_df = pd.DataFrame()

for i in range(5):
    
    train_df =  pd.read_pickle(Path+"train_"+str(i+1)+".csv") 
    val_df   =  pd.read_pickle(Path+"val_"+str(i+1)+".csv") 
    test_df  =  pd.read_pickle(Path+"test_"+str(i+1)+".csv")
    
    train_df = get_function(train_df)
    test_df  = get_function(test_df)
    val_df   = get_function(val_df)
    
    if i == 0:
        predictions1 = np.zeros(len(test_df))
    else:
        pass
    features = feat_common+list(train_df.columns[58:])
    start_time= time.time()
    
    oof = np.zeros(len(val_df))
    max_iter = 5
    
    trn_data = lgb.Dataset(train_df[features],
                               label=train_df['loan_default']
                              )
    val_data = lgb.Dataset(val_df[features],
                               label=val_df['loan_default'] 
                              )

    num_round = 10000
    clf = lgb.train(param, trn_data,
                        num_round,
                        valid_sets = [trn_data, val_data],
                        verbose_eval=100,
                        early_stopping_rounds = 100)
    
    oof = clf.predict(val_df[features], num_iteration=clf.best_iteration)

    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = features
    fold_importance_df["importance"] = clf.feature_importance(importance_type='gain')
    fold_importance_df["fold"] = i + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)

        # we perform predictions by chunks
    current_pred1 = clf.predict(test_df[features].values)
    predictions1 += current_pred

    oof_all.append(oof)
    print(roc_auc_score(val_df['loan_default'].values, oof))
    features = []
predictions1 = predictions1/5


In [34]:
submission = pd.read_csv("test_bqCt9Pv.csv")
submission['loan_default'] = current_pred
submission[['UniqueID','loan_default']].to_csv('Solution_3.csv', index = False)

In [12]:
len(features)

121

In [23]:
train_df[features].dtypes

disbursed_amount                         int64
asset_cost                               int64
ltv                                    float64
Aadhar_flag                              int64
PAN_flag                                 int64
VoterID_flag                             int64
Driving_flag                             int64
Passport_flag                            int64
PERFORM_CNS.SCORE                        int64
PRI.NO.OF.ACCTS                          int64
PRI.ACTIVE.ACCTS                         int64
PRI.OVERDUE.ACCTS                        int64
PRI.CURRENT.BALANCE                      int64
PRI.SANCTIONED.AMOUNT                    int64
PRI.DISBURSED.AMOUNT                     int64
SEC.NO.OF.ACCTS                          int64
SEC.ACTIVE.ACCTS                         int64
SEC.OVERDUE.ACCTS                        int64
SEC.CURRENT.BALANCE                      int64
SEC.SANCTIONED.AMOUNT                    int64
SEC.DISBURSED.AMOUNT                     int64
PRIMARY.INSTA